In [1]:
# Install a bunch of libraries
!pip install neat-python
!pip install gym-retro
!pip install numpy
!pip install cloudpickle
!pip install opencv-python


  Using cached https://files.pythonhosted.org/packages/96/6f/e7074d9c869174c9b7379bd5820f8a774382937053a92c024f0a159e3e05/neat_python-0.92-py3-none-any.whl
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 158.7MB 457kB/s eta 0:00:01 4% |█▋                              | 7.8MB 18.4MB/s eta 0:00:09    33% |██████████▉                     | 53.8MB 18.1MB/s eta 0:00:06    64% |████████████████████▊           | 102.9MB 19.2MB/s eta 0:00:03    93% |██████████████████████████████  | 148.6MB 18.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d4/22/4ff09745ade385ffe707fb5f053548f0f6a6e7d5e98a2b9d6c07f5b931a7/gym-0.10.9.tar.gz
  Using cached https://files.pythonhosted.org/packages/80/39/066ecde98f373430bf7a39a02d91c7075b01ef4fc928456e8e31577342d6/scipy-1.2.0-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.or

In [2]:
# ow you'll need to import all the libraries
import numpy as np
import retro
import neat
import pickle
import cv2

In [3]:
 # now you'll need to import your roms
# use python -m retro.import and point it at the folder with your roms. 
# shasum's of the games need to match those in 
!python -m retro.import ~/games/Roms/Genesis

Importing 10 potential games...
Importing SonicTheHedgehog3-Genesis
Imported 1 games


In [4]:
# Create the environment: Name of folder where rom has been imported, Name of state, where to put recordings
env = retro.make("SonicTheHedgehog3-Genesis", "Level1") 
# if you want training to create bk2 files that you can convert into mp4 vidoes using 
# this command in a termianl: python -m retro.scripts.playback_movie <name-of-some-bk2-file.bk2>, use
# the following env line instead
# env = retro.make("SonicTheHedgehog3-Genesis", "Level1", record='.') 

In [5]:
# This is function that calculates the fitness of each genome in the neat-python Population

def eval_genomes(genomes, config):
    imgarray = []
    xpos_end = 0

# It loops through each genome in the genomes popuation (created in the next cell)
    for genome_id, genome in genomes:
        ob = env.reset()
        ac = env.action_space.sample()
#         print(ob.shape)
        iny, inx, inc = env.observation_space.shape
        inx = int(inx/8)
        iny = int(iny/8)
# this line creates the kind of network you to use. The other main option is:
# net = neat.nn.FeedForwardNetwork.create(genome, config)

        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)
        
# initialize some variables
        current_max_fitness = 0
        fitness_current = 0
        frame = 0
        counter = 0
        xpos = 0
        xpos_max = 0
        
        done = False

        while not done:            

            frame += 1
            ob = cv2.resize(ob, (inx, iny))
            ob = cv2.cvtColor(ob, cv2.COLOR_BGR2GRAY)
            ob = np.reshape(ob, (inx,iny))
            
# The following line lets you watch while training. Commenting it out will speed things up,
# but you wont be able see while it learns. Either way, you can still use playback.py to watch 
# the 'winner.pkl' file after training has completed. 
            env.render()

            imgarray = np.ndarray.flatten(ob)
            
            nnOutput = net.activate(imgarray)
            
            ob, rew, done, info = env.step(nnOutput)

# Set the variables you're tracking for reward
# With Sonic 3, we're only track 'x'
# You can create/change variables using gym-retro-integration (https://www.youtube.com/watch?v=eijRTLh6SBw )

            xpos = info['x']       

# This gives Sonic +1 fitness for every pixel he moves to the right. this is your fitness function.
# This determines how your AI will learn and you can change it however you please!
            if xpos > xpos_max:
                fitness_current += 1
                xpos_max = xpos
            
# This section determines if the AI has finished a level. If it passes that x position, 100000 is
# added to this current genomes fitness, which triggers the network to finish. 

            if xpos == 12300:
                fitness_current += 100000
                done = True

# if you're not interested in manually setting the fitness function, you can just use += rew
           # fitness_current += rew
            
            if fitness_current > current_max_fitness:
                current_max_fitness = fitness_current
                counter = 0
            else:
                counter += 1
                
            if done or counter == 250:
                done = True
                print("Genome: ", genome_id, ", Fitness Achieved: ", fitness_current)
                
            genome.fitness = fitness_current

In [6]:
# download config-feedfoward, the neat-python configuration
!wget http://gitlab.com/lucasrthompson/Sonic-Bot-In-OpenAI-and-NEAT/raw/master/config-feedforward
    

URL transformed to HTTPS due to an HSTS policy
--2018-12-28 10:37:39--  https://gitlab.com/lucasrthompson/Sonic-Bot-In-OpenAI-and-NEAT/raw/master/config-feedforward
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving gitlab.com (gitlab.com)... 35.231.145.151
Connecting to gitlab.com (gitlab.com)|35.231.145.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2060 (2.0K) [text/plain]
Saving to: ‘config-feedforward’

config-feedforward  100%[===================>]   2.01K  --.-KB/s    in 0s      

2018-12-28 10:37:39 (55.6 MB/s) - ‘config-feedforward’ saved [2060/2060]



In [7]:
#Setup config
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config-feedforward')
# creat population
p = neat.Population(config)

# add reporters so you can get some nice stats
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)

# save a check point file every 10 iterations
p.add_reporter(neat.Checkpointer(10))

# if you have already trained some, and want to restore, uncomment this line and change the 'neat-checkpoint-10'
# to whichever file you want to use (they'll all be named neat-checkpoint-somenumber)
#p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-10')

# this line runs the previous eval_genomes function. Once done, the best is set to winner
winner = p.run(eval_genomes)

#saves a pickle file of the winning genome.
with open('winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)


 ****** Running generation 0 ****** 



/home/lucas/code/tutorials/lib/python3.6/site-packages/neat/aggregations.py:18: RuntimeWarning: overflow encountered in double_scalars
  return reduce(mul, x, 1.0)
/home/lucas/code/tutorials/lib/python3.6/site-packages/neat/aggregations.py:18: RuntimeWarning: invalid value encountered in double_scalars
  return reduce(mul, x, 1.0)


Genome:  1 , Fitness Achieved:  368
Genome:  2 , Fitness Achieved:  1


KeyboardInterrupt: 

In [ ]:
# playback winning genome